In [1]:
%load_ext magic_duckdb

In [2]:
%%dql -t df
INSTALL httpfs;

,Success


In [3]:
%%dql -t df
LOAD  httpfs;

,Success


# Data Check

In [ ]:
%%dql -t df
--  Check the tables
SELECT * FROM modealto.order_status
ORDER BY delivered_date DESC
LIMIT 5;

SELECT * FROM modealto.checkout_attempts
ORDER BY checkout_completed_at DESC
LIMIT 5;

# Non-Medication Items Add-to-Cart Rate


In [ ]:
%%dql -t df
SELECT 
  (COUNT(case when customer_added_non_med_to_order = TRUE THEN 1 else NULL END)+0.0)/COUNT(*)*100 as non_med_added_rate
FROM 
  modealto.checkout_attempts
WHERE 
  checkout_completed_at IS NOT NULL;

In [ ]:
%%dql -t df
SELECT 
  EXTRACT(YEAR FROM checkout_started_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_started_at) as check_out_month,
  (COUNT(case when customer_added_non_med_to_order = TRUE THEN 1 else NULL END)+0.0)/COUNT(*)*100 as non_med_added_rate
FROM 
  modealto.checkout_attempts
WHERE 
  checkout_completed_at IS NOT NULL
GROUP BY 
  EXTRACT(YEAR FROM checkout_started_at), 
  EXTRACT(MONTH FROM checkout_started_at)
ORDER BY 
  EXTRACT(YEAR FROM checkout_started_at), 
  EXTRACT(MONTH FROM checkout_started_at)

In [ ]:
%%dql -t df
SELECT 
    ROUND((COUNT(checkout_completed_at) + 0.0) / COUNT(*) * 100, 2) AS check_out_rate
FROM 
    modealto.checkout_attempts;

In [ ]:
%%dql -t df
SELECT 
  EXTRACT(YEAR FROM checkout_started_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_started_at) as check_out_month,
  ROUND((COUNT(checkout_completed_at)+0.0)/COUNT(*)*100,2) as check_out_rate
FROM 
  modealto.checkout_attempts
GROUP BY 
  EXTRACT(YEAR FROM checkout_started_at),
  EXTRACT(MONTH FROM checkout_started_at) 
ORDER BY 
  EXTRACT(YEAR FROM checkout_started_at),
  EXTRACT(MONTH FROM checkout_started_at) 

In [ ]:
%%dql -t df
-- Delivery Rate
SELECT 
    ROUND((COUNT(delivered_date)+0.0)/COUNT(*)*100,2) AS delivery_rate
FROM 
  modealto.checkout_attempts ca
JOIN 
  modealto.order_status os
USING (order_id)

In [ ]:
%%dql -t df
-- Delivery Rate Break Down By Month
SELECT 
  EXTRACT(YEAR FROM checkout_completed_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_completed_at) as check_out_month,
  ROUND((COUNT(CASE WHEN delivery_status = 'delivered' THEN 1 ELSE NULL END)+0.0)/COUNT(*)*100,2) AS delivery_rate
FROM 
  modealto.checkout_attempts ca
JOIN 
  modealto.order_status os
USING (order_id)
GROUP BY 
  EXTRACT(YEAR FROM checkout_completed_at),
  EXTRACT(MONTH FROM checkout_completed_at) 
ORDER BY 
  EXTRACT(YEAR FROM checkout_completed_at),
  EXTRACT(MONTH FROM checkout_completed_at) 

In [ ]:
%%dql -t df
SELECT  
  AVG(CASE WHEN customer_added_non_med_to_order = TRUE THEN order_total_dollars ELSE NULL END) as non_med_added_avg_order_value,
  AVG(CASE WHEN customer_added_non_med_to_order = FALSE THEN order_total_dollars ELSE NULL END) as non_med_not_added_avg_order_value
FROM 
  modealto.checkout_attempts ca
JOIN 
  modealto.order_status os
USING(order_id)
WHERE
  delivery_status = 'delivered';

# Revenue and Number of Orders

In [ ]:
%%dql -t df
SELECT  
  EXTRACT(YEAR FROM checkout_completed_at) as check_out_year, 
  EXTRACT(MONTH FROM checkout_completed_at) as check_out_month,
  SUM(order_total_dollars) as Revenue, 
  COUNT(*) as Number_of_Order
FROM 
  modealto.checkout_attempts ca
JOIN 
  modealto.order_status os
USING(order_id)
WHERE
  delivery_status = 'delivered'
GROUP BY 
  EXTRACT(YEAR FROM checkout_completed_at),
  EXTRACT(MONTH FROM checkout_completed_at) 
ORDER BY 
  EXTRACT(YEAR FROM checkout_completed_at),
  EXTRACT(MONTH FROM checkout_completed_at) 